# All Sports Data Collection

- Data collected from 20 popular sports subreddits
- Max 1000 posts collected from each, due to Reddit's limit on viewable posts

### Imports

In [1]:
import pandas as pd

import requests
import requests.auth
import json

import time
from time import sleep

### Get Authorization

In [2]:
# request a token
client_auth = requests.auth.HTTPBasicAuth('ZfnXA82bMZkXWBRp_29tqw', 'MTr-tnMjbizJ9jeuPQI8EK-CiPUFWA')
post_data = {'grant_type': 'password', 'username': 'gtaylor38gadsi', 'password': 'axfig654$Bin'}
headers = {'User-Agent': 'PostScraperScript/1.0 by gtaylor38gadsi'}
response_token = requests.post('https://www.reddit.com/api/v1/access_token', auth=client_auth, data=post_data, headers=headers)
response_token.json()

{'access_token': '2205940059860-tmuQaSyS5zLBJ-Ft1wgn_uif033vlw',
 'token_type': 'bearer',
 'expires_in': 86400,
 'scope': '*'}

In [3]:
# store credentials
access_token = response_token.json()['access_token']
token_type = response_token.json()['token_type']

In [4]:
# update headers for data collection
headers = {"Authorization": f'{token_type} {access_token}', 'User-Agent': 'PostScraperScript/1.0 by gtaylor38gadsi'}

### Collect Data

In [5]:
# initialize df
posts_df = pd.DataFrame(columns=['title', 'subreddit', 'score', 'author_flair_text', 'link_flair_text',
                                 'created_utc', 'num_comments', 'age'])

# define columns to collect
base_cols = ['title', 'subreddit', 'score', 'author_flair_text', 'link_flair_text', 'created_utc', 'num_comments']

# create subreddit list
subreddits = ['sports', 'nba', 'soccer', 'nfl', 'baseball',
              'collegebasketball', 'mma', 'formula1', 'running', 'hockey',
              'cfb', 'snowboarding', 'skiing', 'boxing', 'bicycling',
              'climbing', 'fishing', 'nascar', 'tennis', 'golf']

In [6]:
# collect ~1000 posts from each subreddit
for sub in subreddits:
    # set base url
    url_base = f'https://oauth.reddit.com/r/{sub}/new/.json'

    # make initial request, initialize page list, define columns to scrape
    response = requests.get(url_base, headers=headers)
    page_new = []

    # get data from first page using base url and store in posts_df
    for i in range(response.json()['data']['dist']):
        row = [response.json()['data']['children'][i]['data'][col_name] for col_name in base_cols]
        page_new.append(row)
    df = pd.DataFrame(page_new, columns=base_cols)
    curr_time = time.time()
    df['age'] = curr_time - df['created_utc']
    posts_df = pd.concat([posts_df, df])

    # collect data from next 39 pages
    for i in range(39):
        # update after and url_current
        after = response.json()['data']['after']
        url_current = f'{url_base}?after={after}'

        # make new request, update time, and reset page list
        response = requests.get(url_current, headers=headers)
        page_new = []

        # collect and store data in page list
        for j in range(response.json()['data']['dist']):
            row = [response.json()['data']['children'][j]['data'][col_name] for col_name in base_cols]
            page_new.append(row)

        # store page list as df, add age column, and concat df to posts_df
        df = pd.DataFrame(page_new, columns=base_cols)
        curr_time = time.time()
        df['age'] = curr_time - df['created_utc']
        posts_df = pd.concat([posts_df, df])
        
    # sleep one minute to comply with requests limit
    sleep(60)

In [7]:
# check data
print('posts_df shape:', posts_df.shape, '\n# of unique posts:', len(posts_df.title.unique()))

posts_df shape: (19776, 8) 
# of unique posts: 18478


In [11]:
posts_df.subreddit.value_counts()

MMA                  999
tennis               998
snowboarding         996
soccer               995
CollegeBasketball    995
Fishing              993
running              993
hockey               993
CFB                  992
NASCAR               991
golf                 991
nfl                  990
bicycling            989
formula1             989
Boxing               986
baseball             986
nba                  985
skiing               985
climbing             980
sports               950
Name: subreddit, dtype: int64

### Save Data

In [12]:
# Write posts_df to csv:
posts_df.to_csv('data/all_sports_posts.csv', index=False)